#### 1. **Model Runner Step Sync And Async Testing**

Notebook tests `ModelRunnerStep` within a serving graph to enable real-time monitoring and drift detection with MLRun. 
The main focus is to test te preformance of sync and async usage.

In [1]:
# Import mlrun and create project instance
import mlrun

image = "mlrun/mlrun"
project_name = "monitored-model-runner"
project = mlrun.get_or_create_project(project_name, context="./",user_project=True)

> 2025-10-26 10:01:25,401 [info] Project loaded successfully: {"project_name":"monitored-model-runner-matanz"}


In [2]:
# Import tools
import pandas as pd
from sklearn.svm import SVC
import pickle
from sklearn.datasets import load_iris
from mlrun.features import Feature

In [17]:
# Train simple SVM model on Iris dataset, save it and reformat the DS as list
iris = load_iris()
clf = SVC()           
clf.fit(iris.data, iris.target)
with open("SVM.pkl", "wb") as fh:
    pickle.dump(clf, fh)
iris_data = iris["data"].tolist()

# load the dataset again as a DF
iris = load_iris()
train_set = pd.DataFrame(
    iris["data"],
    columns=["sepal_length_cm", "sepal_width_cm", "petal_length_cm", "petal_width_cm"],
)

# Create a Model Artifact in the project using the trained model
model_name = "SVM"
model_artifact = project.log_model(
    model_name,
    model_file="SVM.pkl",
    training_set=train_set,
    framework="sklearn",
    outputs=[Feature(name="label")],
)

#### 5. Define your function and ModelRunnerStep

Define functions to all the edge cases

In [28]:
# Config the code path and the serving function sync and async
sync_code_path = r"model_class_sync.py"
bouth_code_path = r"model_class_bouth.py"


sync_function = project.set_function(func=sync_code_path,image="mlrun/mlrun",kind="serving",name="serving-function")
async_sync_function = project.set_function(func=sync_code_path,image="mlrun/mlrun",kind="serving",name="serving-function")
bouth_function = project.set_function(func=bouth_code_path,image="mlrun/mlrun",kind="serving",name="serving-function")

In [29]:
from mlrun.serving.states import ModelRunnerStep

model_runner_step = ModelRunnerStep(
    name="my_runner", model_selector="MyModelSelector",model_selector_parameters={"name":"my-selector"})

model_runner_step.add_model(
    model_class="MyModel",
    endpoint_name="my-model",
    model_artifact=model_artifact,
    input_path="inputs.here",
    result_path="outputs",
    outputs=["label"],
    execution_mechanism="thread_pool",
)

for i in range(10):
    model_runner_step.add_model(
        model_class="MyModel",
        endpoint_name=f"my-{i}-model",
        model_artifact=model_artifact,
        input_path="inputs.here",
        result_path="outputs",
        outputs=["label"],
        execution_mechanism="thread_pool",
    )

#### 6. Build graphs to all the edge cases

description


In [31]:
sync_graph = sync_function.set_topology("flow",engine="sync")
sync_graph.to("MyPreprocessStep").to(model_runner_step).to("MyEnrichStep").respond()
sync_graph.plot()

async_graph_defined = bouth_function.set_topology("flow",engine="async")
async_graph_defined.to("MyPreprocessStep").to(model_runner_step).to("MyEnrichStep").respond()
async_graph_defined.plot()

async_graph_undefined = async_sync_function.set_topology("flow",engine="async")
async_graph_undefined.to("MyPreprocessStep").to(model_runner_step).to("MyEnrichStep").respond()
async_graph_undefined.plot()


MLRunInvalidArgumentError: graph topology is already set, cannot be overwritten